TP5:

In [254]:
import pandas as pd 
import numpy as np 
import math 

In [255]:
def load(path):
 data = pd.read_csv(path, delimiter=',')
 
 return data
data=load("DatasetExos.csv")

In [256]:
# data['ep (ms)'] = pd.to_datetime(data['ep (ms)'])
data['ep (ms)'] =  data['ep (ms)'] .astype('category')  

data['Acc_x'] = pd.to_numeric(data['Acc_x'], errors='coerce') 
data['Acc_y'] = pd.to_numeric(data['Acc_y'], errors='coerce')  
data['Acc_z'] = pd.to_numeric(data['Acc_z'], errors='coerce')  
data['Gyro_x'] = pd.to_numeric(data['Gyro_x'], errors='coerce')  
data['Gyro_y'] = pd.to_numeric(data['Gyro_y'], errors='coerce')  
data['Gyro_z'] = pd.to_numeric(data['Gyro_z'], errors='coerce')  
data['ID'] = data['ID'].astype('category')  
data['Exercise'] = data['Exercise'].astype('category')  
data['Category'] = data['Category'].astype('category')  
data['Set'] = data['Set'].where(data['Set'] >= 0,0) 
data['Set'] = data['Set'].fillna(0)  
data['Set']= pd.to_numeric(data['Set'], errors='coerce') 

print(data.dtypes)

ep (ms)     category
Acc_x        float64
Acc_y        float64
Acc_z        float64
Gyro_x       float64
Gyro_y       float64
Gyro_z       float64
ID          category
Exercise    category
Category    category
Set          float64
dtype: object


In [257]:
def min_max_normalize(data):
   
    
    for column in data.select_dtypes(include=[np.number]).columns:
        min_val = data[column].min()
        max_val = data[column].max()
        newmin=0
        newmax=1
        data[column] =((data[column] - min_val)*(newmax-newmin)/ (max_val - min_val))+newmin
    
    return data

In [258]:
def remove_outliers(data, threshold=1.5):
    """
    Remove outliers based on the IQR method.
    """
    numerical_data = data.select_dtypes(include=[np.number])
    
    # Apply the IQR method to each numerical column
    for column in numerical_data.columns:
        Q1 = np.percentile(numerical_data[column], 25)
        Q3 = np.percentile(numerical_data[column], 75)
        IQR = Q3 - Q1
        
        # Define lower and upper bounds
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        # Remove outliers from the numerical column
        # data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]=np.nan
    
    return data


In [259]:


# Filter out rows where data is outside the bounds
data=remove_outliers(data)



In [260]:
data = data.apply(
    lambda x: x.fillna(x.mode()[0]) if x.dtype == 'category' and x.mode().size > 0 else x.fillna(x.mean()) if x.dtype != 'category' else x,
    axis=0
)

In [261]:
print("number missing values" ,data.isnull().sum())


number missing values ep (ms)     0
Acc_x       0
Acc_y       0
Acc_z       0
Gyro_x      0
Gyro_y      0
Gyro_z      0
ID          0
Exercise    0
Category    0
Set         0
dtype: int64


In [262]:
data=min_max_normalize(data)

In [263]:
data = data.drop(columns=['Exercise','ID','ep (ms)','Category'])

In [264]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

def distance(x, y, data):
    # Convert x and y to pandas Series if they aren't already
    x = pd.Series(x)
    y = pd.Series(y)
    
    # Identify numerical and categorical columns
    numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns
    
    # Get indices of numerical and categorical columns
    numerical_indices = [data.columns.get_loc(col) for col in numerical_columns]
    categorical_indices = [data.columns.get_loc(col) for col in categorical_columns]
    
    # Extract numerical values
    numerical_X = x.iloc[numerical_indices].values.flatten()
    numerical_Y = y.iloc[numerical_indices].values.flatten()
    
    # Calculate Manhattan distance for numerical columns
    manhattan = np.sum(np.abs(numerical_X - numerical_Y))
    
    # # Extract categorical values
    # categorical_X = x.iloc[categorical_indices].values.flatten()
    # categorical_Y = y.iloc[categorical_indices].values.flatten()
    
    # # Calculate Hamming distance for categorical columns
    # mismatches = np.sum(categorical_X != categorical_Y)
    # if (categorical_X==0):
    #     categorical_X=1
    # hamming = mismatches / len(categorical_X)
    
    # Weighted combination of distances
    return manhattan 

def initialize_centroids(data, k):
    """
    Initialize centroids by randomly selecting k unique instances from the data.
    
    :param data: DataFrame
    :param k: int (number of clusters)
    :return: DataFrame of centroids
    """
    return data.sample(n=k, replace=False).reset_index(drop=True)

def calculate_centroid(cluster_data):
    """
    Calculate the centroid of a cluster.
    
    :param cluster_data: DataFrame of cluster points
    :return: Centroid (Series)
    """
    # For numerical columns, take mean
    numerical_cols = cluster_data.select_dtypes(include=['int64', 'float64']).columns
    
    # For categorical columns, take mode
    categorical_cols = cluster_data.select_dtypes(include=['object', 'category']).columns
    
    # Create centroid
    centroid = pd.concat([
        cluster_data[numerical_cols].mean(),
        cluster_data[categorical_cols].mode().iloc[0]
    ])
    
    return centroid

def closest_cluster(instance, centroids, data):
    """
    Find the closest cluster for a given instance.
    
    :param instance: Single data point
    :param centroids: Centroids of clusters
    :param data: Original dataset
    :return: Index of the closest cluster
    """
    distances = [distance(c, instance, data) for c in centroids.values]
    return np.argmin(distances)

def k_means(data, k, max_iters=100):
    """
    Perform K-means clustering.
    
    :param data: DataFrame to cluster
    :param k: Number of clusters
    :param max_iters: Maximum number of iterations
    :return: Clustered data and final centroids
    """
    # Preprocess data
    # Separate features and categorical columns
    numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns
    
    # Standardize numerical columns
    scaler = StandardScaler()
    data_scaled = data.copy()
    data_scaled[numerical_cols] = scaler.fit_transform(data[numerical_cols])
    
    # Initialize centroids
    centroids = initialize_centroids(data_scaled, k)
    
    for _ in range(max_iters):
        # Assign clusters
        cluster_assignments = data_scaled.apply(
            lambda row: closest_cluster(row, centroids, data_scaled), 
            axis=1
        )
        data['cluster'] = cluster_assignments
        
        # Recalculate centroids
        new_centroids = cluster.mean(axis=0)
        
        # Check for convergence
        if centroids.equals(new_centroids):
            print('Convergence reached.')
            break
        
        centroids = new_centroids
    
    return data, centroids

def evaluate_clusters(data, cluster_assignments):
    """
    Evaluate clustering performance.
    
    :param data: Original dataset
    :param cluster_assignments: Cluster labels
    :return: WCSS and Silhouette score
    """
    # Prepare data for evaluation (use original numerical columns)
    numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
    data_for_eval = data[numerical_cols]
    
    # Calculate Within-Cluster Sum of Squares (WCSS)
    wcss = np.sum([
        np.sum((data_for_eval[cluster_assignments == c] - 
                data_for_eval[cluster_assignments == c].mean())**2)
        for c in np.unique(cluster_assignments)
    ])
    
    # Calculate Silhouette Score
    silhouette_avg = silhouette_score(data_for_eval, cluster_assignments)
    
    return wcss, silhouette_avg

def visualize_with_pca(data, cluster_assignments, title):
    """
    Visualize clusters using PCA.
    
    :param data: Original dataset
    :param cluster_assignments: Cluster labels
    :param title: Plot title
    """
    # Prepare data for PCA (use only numerical columns)
    numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
    data_for_pca = data[numerical_cols]
    
    # Reduce data to 2D using PCA
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data_for_pca)
    
    # Visualize clusters
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(data_2d[:, 0], data_2d[:, 1], 
                          c=cluster_assignments, 
                          cmap='viridis', 
                          s=30, 
                          alpha=0.6)
    plt.title(title)
    plt.xlabel('PC1 (1st principal component)')
    plt.ylabel('PC2 (2nd principal component)')
    plt.colorbar(scatter, label='Cluster')
    plt.tight_layout()
    plt.show()

# Example usage (modify according to your specific dataset)

    # Assume 'data' is your preprocessed DataFrame
for k in [2, 3, 4, 5]:
        # Clustering
        clustered_data, centroids = k_means(data, k)
        
        # Evaluate clusters
        wcss, silhouette_avg = evaluate_clusters(data, clustered_data['cluster'])
        
        # Visualize with PCA
        visualize_with_pca(data, clustered_data['cluster'], f"K-Means with k={k}")
        
        # Print evaluation results
        print(f"Results for k={k}:")
        print(f"- Intra-cluster inertia (WCSS): {wcss:.2f}")
        print(f"- Silhouette score: {silhouette_avg:.2f}")

# Uncomment and run when you have your dataset ready
# if __name__ == "__main__":
#     main()

IndexError: single positional indexer is out-of-bounds

In [228]:
import numpy as np

def distance(x, y, data):
    numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns
    # Get indices of numerical columns
    numerical_indices = [data.columns.get_loc(col) for col in numerical_columns]

    categorical_indices = [data.columns.get_loc(col) for col in categorical_columns]
    x = pd.Series(x)
    y = pd.Series(y)
    
    # Extract numerical and categorical values for x and y
    numerical_X = x.iloc[numerical_indices].values
    numerical_Y = y.iloc[numerical_indices].values
    numerical_X  = [item[0] for item in  numerical_X ]
    # Calculate Manhattan distance for numerical columns
    manhattan = sum(abs(numerical_X - numerical_Y))
    
    categorical_X = x.iloc[categorical_indices].values
    categorical_Y = y.iloc[categorical_indices].values
    categorical_X  = [item[0] for item in  categorical_X ]
    
    # Calculate Hamming distance for categorical columns
    mismatches = sum(a != b for a, b in zip(categorical_X, categorical_Y))
    hamming = mismatches / len(categorical_X)
    
    # return ((len(numerical_columns) / len(data.columns)) * manhattan) + ((len(categorical_columns) / len(data.columns)) * hamming)
    return ( manhattan) + (  hamming)



In [229]:
# def sortedval(data, x ,pred):
#     d=[]
    
#     # Drop the prediction column once to avoid redundant computation
#     data_dropped = data.drop(columns=[pred])
    
#     # Iterate through each instance in the dataset
#     for _, row in data.iterrows():
#         dist = distance(x, row.drop(pred), data_dropped)
#         d.append((dist, row[pred]))
    
#     # Sort the list by the calculated distance
#     d.sort(key=lambda item: item[0])
    
#     return d
    
    

In [230]:
# from collections import Counter

# def maxclass(d,k):
#   _, l = zip(*d[:k])
#   class_counts = Counter(l)
    
#   dominant, frequency = class_counts.most_common(1)[0]
#   return dominant
    
    

In [231]:
# def knn(k,data,pred,x):
#    d= sortedval(data, x ,pred)
#    return maxclass(d,k)
   
    

In [232]:
# instance =['2024-11-20 18:09:51.000', -0.137, 1.066, 0.8215, -6.597, 0.808, 1.985, 'B', 'medium', 30]
# typed_instance = [(i, type(i)) for i in instance]
# print ('pour k=3')
# print(knn(3,data,'Exercise',typed_instance))
# print ('pour k=10')
# print(knn(10,data,'Exercise',typed_instance))

Tp6

In [233]:
data = data.drop(columns=['Exercise','ID','ep (ms)','Category'])

In [265]:
def initialize_centroids(data, k):
    """
    Initialize centroids by randomly selecting k unique instances from the data.
    :param data: numpy array of shape (n_instances, n_features)
    :param k: int (number of clusters)
    :return: numpy array of shape (k, n_features)
    """
    centroids = []  # List to store the centroids
    for _ in range(k):  # Loop k times to select k centroids
     centroids.append(data.sample(replace=False).iloc[0])  # Randomly select a single instance and add it to centroids
    return pd.DataFrame(centroids).reset_index(drop=True)

In [266]:
def closest_cluster(instance, centroids,data):
    distances=[]
    for c in centroids:
     d = distance(c, instance, data)
     distances.append((d))
    
#     # Sort the list by the calculated distance
#     d.sort(key=lambda item: item[0])

    return np.argmin(distances)

In [267]:
def calculate_centroid(cluster):
    return cluster.mean(axis=0)

In [268]:
def k_means(data, k, max_iters=100):
    """
    K-means algorithm that stops when all clusters contain the same number of instances.
    :param data: numpy array of shape (n_instances, n_features)
    :param k: int (number of clusters)
    :param max_iters: int (maximum number of iterations)
    :return: (numpy array of cluster assignments, numpy array of centroids)
    """
    # Step 1: Initialize centroids randomly from the data
    centroids = initialize_centroids(data, k)

    for _ in range(max_iters):
        # Step 2: Assign each instance to the nearest centroid
        cluster_assignments = np.array([closest_cluster(instance, centroids,data) for instance in data])

        # Step 3: Count the number of instances in each cluster
        _, counts = np.unique(cluster_assignments, return_counts=True)

        # Step 4: Check if all clusters have the same number of instances
        if len(set(counts)) == 1:  # If all counts are equal
            break

        # Step 5: Update centroids
        centroids = np.array([calculate_centroid(data[cluster_assignments == c]) for c in range(k)])

    return cluster_assignments, centroids

In [269]:
# from sklearn.metrics import silhouette_score
# import matplotlib.pyplot as plt
# from sklearn.datasets import make_blobs


# # def evaluate_clusters(data, cluster_assignments):
# #     wcss = np.sum([np.linalg.norm(data[cluster_assignments == c] - np.mean(data[cluster_assignments == c], axis=0))**2 
# #                    for c in np.unique(cluster_assignments)])
# #     silhouette_avg = silhouette_score(data, cluster_assignments)
# #     return wcss, silhouette_avg

In [270]:
from sklearn.decomposition import PCA

# Fonction pour visualiser avec PCA
def visualize_with_pca(data, cluster_assignments, centroids, title):
    # Réduire les données en 2 dimensions avec PCA
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data)
    centroids_2d = pca.transform(centroids)
    
    # Visualisation des clusters
    plt.scatter(data_2d[:, 0], data_2d[:, 1], c=cluster_assignments, cmap='viridis', s=30, alpha=0.6)
    plt.scatter(centroids_2d[:, 0], centroids_2d[:, 1], color='red', marker='x', s=200, label='Centroids')
    plt.title(title)
    plt.xlabel('PC1 (1ère composante principale)')
    plt.ylabel('PC2 (2ème composante principale)')
    plt.legend()
    plt.show()

# Tester avec PCA pour visualisation
for k in [2, 5, 6]:
    cluster_assignments, centroids = k_means(data, k)
    wcss, silhouette_avg = evaluate_clusters(data, cluster_assignments)
    
    # Visualisation avec PCA
    visualize_with_pca(data, cluster_assignments, centroids, f"K-Means avec k={k} (Visualisation PCA)")
    
    # Afficher les résultats d'évaluation
    print(f"Résultats pour k={k}:")
    print(f"- Inertie intra-cluster (WCSS) : {wcss:.2f}")
    print(f"- Score de silhouette : {silhouette_avg:.2f}")
# #     print()


IndexError: positional indexers are out-of-bounds

In [240]:
print(data.shape)

(9009, 7)


In [241]:
def k_means(data, k, max_iters=100):
    # Step 1: Initialize centroids randomly from the data
    centroids = initialize_centroids(data, k)
    data['cluster'] = np.random.randint(0, k, size=len(data))
    print(data['cluster'])

    for _ in range(max_iters):
        # Step 2: Assign clusters
        data['cluster'] = data.apply(lambda row: closest_cluster(row[:-1], centroids, data), axis=1)

        # Step 3: Recalculate centroids
        new_centroids = data.groupby('cluster').apply(lambda cluster: calculate_centroid(cluster.iloc[:, :-1])).reset_index(drop=True)

        # Step 4: Check for convergence
        if centroids.equals(new_centroids):
            print('Convergence reached.')
            break

        centroids = new_centroids

    return data, centroids

# Evaluate clusters using WCSS and silhouette score
def evaluate_clusters(data, cluster_assignments):
    wcss = np.sum([np.linalg.norm(data[cluster_assignments == c] - np.mean(data[cluster_assignments == c], axis=0))**2
                   for c in np.unique(cluster_assignments)])
    silhouette_avg = silhouette_score(data, cluster_assignments)
    return wcss, silhouette_avg

# Visualize clusters with PCA
def visualize_with_pca(data, cluster_assignments, centroids, title):
    # Reduce data to 2D using PCA
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data)
    centroids_2d = pca.transform(centroids)

    # Visualize clusters
    plt.scatter(data_2d[:, 0], data_2d[:, 1], c=cluster_assignments, cmap='viridis', s=30, alpha=0.6)
    plt.scatter(centroids_2d[:, 0], centroids_2d[:, 1], color='red', marker='x', s=200, label='Centroids')
    plt.title(title)
    plt.xlabel('PC1 (1st principal component)')
    plt.ylabel('PC2 (2nd principal component)')
    plt.legend()
    plt.show()

# Test with PCA visualization for different k values
for k in [2, 5, 6]:
    data, centroids = k_means(data, k)
    wcss, silhouette_avg = evaluate_clusters(data, data['cluster'])
    
    # Visualize with PCA
    visualize_with_pca(data.iloc[:, :-1], data['cluster'], centroids, f"K-Means with k={k} (PCA Visualization)")
    
    # Print evaluation results
    print(f"Results for k={k}:")
    print(f"- Intra-cluster inertia (WCSS): {wcss:.2f}")
    print(f"- Silhouette score: {silhouette_avg:.2f}")
    print()

0       1
1       1
2       1
3       1
4       0
       ..
9004    0
9005    0
9006    0
9007    1
9008    0
Name: cluster, Length: 9009, dtype: int32


IndexError: positional indexers are out-of-bounds